# Import packages


In [2]:
import numpy as np
import pylab as pl
from numpy import fft
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta
import math


# Load data

In [3]:
def load_data(stock_name, date_predict_start, data_range, slide_range, n_slide):
    train_data = {}
    test_data = {}
    date_predict_start = datetime.datetime.strptime(
        date_predict_start, '%Y-%m-%d')
    date_data_start_list = []
    date_predict_start_list = []
    date_predict_end_list = []
    test_data_start_list = []
    all_data = yf.Ticker(stock_name).history(period='max')

    check = date_predict_start in list(all_data.index)
    while (check == False):
        date_predict_start = date_predict_start + \
            relativedelta(days=+1)
        check = date_predict_start in list(all_data.index)

    predict_start = all_data.index.get_loc(date_predict_start)
    for i in range(n_slide):
        predict_end = predict_start + data_range
        date_predict_end = all_data.iloc[predict_end].name
        data_start = predict_start - data_range
        date_data_start = all_data.iloc[data_start].name
        train_data['data_' + str(i)] = all_data.iloc[data_start:predict_start]
        test_data['data_' + str(i)] = all_data.iloc[predict_start:predict_end]
        date_data_start_list.append(date_data_start)
        date_predict_start_list.append(date_predict_start)
        date_predict_end_list.append(date_predict_end)
        test_data_start_list.append(test_data['data_' + str(i)].index[0])
        data_start = data_start + slide_range
        predict_start = predict_start + slide_range
        train_data['data_' + str(i)] = train_data['data_' +
                                                  str(i)].reset_index(drop=True)
        test_data['data_' + str(i)] = test_data['data_' +
                                                str(i)].reset_index(drop=True)

    return train_data, test_data, all_data, test_data_start_list


# Data preprocessing

In [4]:
def find_data_pv_CL_function(data, pv_range):
    pd.options.mode.chained_assignment = None
    for i in data:
        price = data[i]['Close']
        data[i]['peaks'] = pd.Series(dtype='float64')
        data[i]['valleys'] = pd.Series(dtype='float64')
        data[i]['pv'] = pd.Series(dtype='str')
        for idx in range(0, len(price)):
            if idx < pv_range:
                if price[idx] == price.iloc[0:pv_range*2+1].max():
                    data[i]['peaks'].iloc[idx] = price[idx]
                    data[i]['pv'].iloc[idx] = 'peaks'
                if price[idx] == price.iloc[0:pv_range*2+1].min():
                    data[i]['valleys'].iloc[idx] = price[idx]
                    data[i]['pv'].iloc[idx] = 'valleys'
            if price[idx] == price.iloc[idx-pv_range:idx+pv_range].max():
                data[i]['peaks'].iloc[idx] = price[idx]
                data[i]['pv'].iloc[idx] = 'peaks'
            if price[idx] == price.iloc[idx-pv_range:idx+pv_range].min():
                data[i]['valleys'].iloc[idx] = price[idx]
                data[i]['pv'].iloc[idx] = 'valleys'
    

In [5]:
def find_data_pv_HL_function(data, pv_range):
    pd.options.mode.chained_assignment = None
    for i in data:
        high = data[i]['High']
        low = data[i]['Low']
        data[i]['peaks'] = pd.Series(dtype='float64')
        data[i]['valleys'] = pd.Series(dtype='float64')
        data[i]['pv'] = pd.Series(dtype='str')
        for idx in range(0, len(high)):
            if idx < pv_range:
                if high[idx] == high.iloc[0:pv_range*2+1].max():
                    data[i]['peaks'].iloc[idx] = high[idx]
                    data[i]['pv'].iloc[idx] = 'peaks'
                if low[idx] == low.iloc[0:pv_range*2+1].min():
                    data[i]['valleys'].iloc[idx] = low[idx]
                    data[i]['pv'].iloc[idx] = 'valleys'
            if high[idx] == high.iloc[idx-pv_range:idx+pv_range].max():
                data[i]['peaks'].iloc[idx] = high[idx]
                data[i]['pv'].iloc[idx] = 'peaks'
            if low[idx] == low.iloc[idx-pv_range:idx+pv_range].min():
                data[i]['valleys'].iloc[idx] = low[idx]
                data[i]['pv'].iloc[idx] = 'valleys'

In [6]:
def preprocessing(train_data, test_data, pv_range, pv_method):
    if pv_method == 'CL':
        find_data_pv_CL_function(train_data, pv_range)
        find_data_pv_CL_function(test_data, pv_range)
    elif pv_method == 'HL':
        find_data_pv_HL_function(train_data, pv_range)
        find_data_pv_HL_function(test_data, pv_range)


# Built Model

In [7]:
def data_to_poly_function(data_stock):
    harmonics = {}
    for i in data_stock:
        harmonics[i] = {}
        data = data_stock[i]['Close']
        array_data = np.array(data)
        n_data = array_data.size
        time_data = np.arange(0, n_data)
        poly = np.zeros_like(array_data)
        # fig, axes = plt.subplots(n_data, 1, figsize=(15, n_data*3))
        # axes[0].plot(data)
        for j in range(0, n_data):
            Polynomial = np.polyfit(time_data, array_data, j)
            for k in range(0, j):
                poly += Polynomial[k] * time_data**(j-k) 
            harmonics[i][j] = np.tile(poly,2)
            # axes[j].plot(poly)
    # plt.show()
    return harmonics

In [8]:
def poly_function(harmonics, n_harm_lower_limit, n_harm_upper_limit):
    processed_signal = {}
    for i in harmonics:
        processed_signal[i] = {}
        for n_harm in range(n_harm_lower_limit, n_harm_upper_limit+1):
            processed_signal[i][n_harm] = pd.DataFrame(
                {'Close': harmonics[i][n_harm]})
    return processed_signal


# Built Model

Get signal

In [9]:
def data_to_harmonics_function(data_stock, data_range):
    harmonics = {}
    for i in data_stock:
        harmonics[i] = {}
        # get data_stock's infomation
        data = data_stock[i]['Close']
        array_data = np.array(data)
        n_data = array_data.size
        time_data = np.arange(0, n_data)

        # detrend data
        # find linear trend in data
        Polynomial = np.polyfit(time_data, array_data, 1)
        data_notrend = array_data - Polynomial[0] * time_data    # detrended x

        # fft process
        data_freqdom = fft.fft(data_notrend, n=n_data)
        frequence = fft.fftfreq(n=n_data, d=1)
        f_positive = frequence[np.where(frequence > 0)]
        data_freqdom_positive = data_freqdom[np.where(frequence > 0)]

        # sort indexes
        indexes = list(range(f_positive.size))      # frequencies
        # sort method 1
        # indexes.sort(key = lambda i: np.absolute(frequence[i]))     # sort indexes by frequency, lower -> higher
        # sort method 2 :
        # sort indexes by amplitudes, lower -> higher
        indexes.sort(key=lambda i: np.absolute(data_freqdom[i]))
        indexes.reverse()       # sort indexes by amplitudes, higher -> lower

        # get data_all_time'size
        time_transfer = np.arange(0, data_range*2)

        # mix harmonics
        for j in indexes:
            ampli = np.absolute(
                data_freqdom_positive[j]) / n_data     # amplitude
            phase = np.angle(data_freqdom_positive[j])      # phase
            harmonics[i][j] = ampli * \
                np.cos(2 * np.pi * f_positive[j] * time_transfer + phase)
    return harmonics


In [10]:
def mix_harmonics_function(harmonics, n_harm_lower_limit, n_harm_upper_limit):
    processed_signal = {}
    for i in harmonics:
        processed_signal[i] = {}
        for n_harm in range(n_harm_lower_limit, n_harm_upper_limit+1):
            mixed_harmonic = np.zeros(len(harmonics[i][0]))
            for j in range(n_harm):
                mixed_harmonic += harmonics[i][j]
            processed_signal[i][n_harm] = pd.DataFrame(
                {'Close': mixed_harmonic})
    return processed_signal


Signal processing

In [11]:
def find_signal_pv_function(signal, pv_range):
    pd.options.mode.chained_assignment = None
    for i in signal:
        for j in signal[i]:
            data = signal[i][j]['Close']
            signal[i][j]['peaks'] = pd.Series(dtype='float64')
            signal[i][j]['valleys'] = pd.Series(dtype='float64')
            signal[i][j]['pv'] = pd.Series(dtype='str')
            for idx in range(0, len(data)):
                if idx < pv_range:
                    if data[idx] == data.iloc[0:pv_range*2+1].max():
                        signal[i][j]['peaks'].iloc[idx] = data[idx]
                        signal[i][j]['pv'].iloc[idx] = 'peaks'
                    if data[idx] == data.iloc[0:pv_range*2+1].min():
                        signal[i][j]['valleys'].iloc[idx] = data[idx]
                        signal[i][j]['pv'].iloc[idx] = 'valleys'
                if data[idx] == data.iloc[idx-pv_range:idx+pv_range].max():
                    signal[i][j]['peaks'].iloc[idx] = data[idx]
                    signal[i][j]['pv'].iloc[idx] = 'peaks'
                if data[idx] == data.iloc[idx-pv_range:idx+pv_range].min():
                    signal[i][j]['valleys'].iloc[idx] = data[idx]
                    signal[i][j]['pv'].iloc[idx] = 'valleys'

In [12]:
def find_pv_lead_function(data, processed_signal):
    for d in data:
        for p in processed_signal[d]:
            # processed_signal[d][p]['pv'] = pd.Series(dtype='str')
            processing_signal = processed_signal[d][p].head(len(data[d]))
            p_data = pd.DataFrame(
                {'peaks': data[d]['peaks'], 'count': range(len(data[d]))})
            p_data = p_data.drop(p_data[p_data['peaks'].isna()].index)
            p_data_count = list(p_data['count'])
            p_signal = pd.DataFrame(
                {'peaks': processing_signal['peaks'], 'count': range(len(processing_signal))})
            p_signal = p_signal.drop(p_signal[p_signal['peaks'].isna()].index)
            p_signal_list = list(p_signal['count'])
            p_lead = []
            for i in range(0, len(p_signal_list)):
                temp = []
                temp_abs = []
                temp_2 = []
                for j in range(0, len(p_data_count)):
                    temp.append((p_data_count[j] - p_signal_list[i]))
                    temp_abs.append(abs(p_data_count[j] - p_signal_list[i]))
                for k in range(0, len(temp_abs)):
                    if temp_abs[k] == min(temp_abs):
                        temp_2 = temp[k]
                p_lead.append(temp_2)
            p_signal['lead'] = p_lead

            v_data = pd.DataFrame(
                {'valleys': data[d]['valleys'], 'count': range(len(data[d]))})
            v_data = v_data.drop(v_data[v_data['valleys'].isna()].index)
            v_data_count = list(v_data['count'])
            v_signal = pd.DataFrame(
                {'valleys': processing_signal['valleys'], 'count': range(len(processing_signal))})
            v_signal = v_signal.drop(
                v_signal[v_signal['valleys'].isna()].index)
            v_signal_list = list(v_signal['count'])
            v_lead = []
            for i in range(0, len(v_signal_list)):
                temp = []
                temp_abs = []
                temp_2 = []
                for j in range(0, len(v_data_count)):
                    temp.append((v_data_count[j] - v_signal_list[i]))
                    temp_abs.append(abs(v_data_count[j] - v_signal_list[i]))
                for k in range(0, len(temp_abs)):
                    if temp_abs[k] == min(temp_abs):
                        temp_2 = temp[k]
                v_lead.append(temp_2)
            v_signal['lead'] = v_lead

            processed_signal[d][p]['lead'] = pd.Series(dtype='float64')
            processed_signal[d][p]['lead'].loc[p_signal['lead'].index] = p_signal['lead']
            # processed_signal[d][p]['pv'].loc[p_signal['lead'].index] = 'peak'
            processed_signal[d][p]['lead'].loc[v_signal['lead'].index] = v_signal['lead']
            # processed_signal[d][p]['pv'].loc[v_signal['lead'].index] = 'valley'


In [13]:
def build_model(train_data, n_harm_lower_limit, n_harm_upper_limit, pv_range, data_range):
    # harmonics = data_to_poly_function(train_data)
    # processed_signal = poly_function(harmonics, n_harm_lower_limit, n_harm_upper_limit)
    harmonics = data_to_harmonics_function(train_data, data_range)
    processed_signal = mix_harmonics_function(
        harmonics, n_harm_lower_limit, n_harm_upper_limit)
    find_signal_pv_function(processed_signal, pv_range)
    find_pv_lead_function(train_data, processed_signal)
    return harmonics, processed_signal


# Select model

In [14]:
def get_fit_error_function(processed_signal, fit_method):
    errors = {}
    error = []
    for i in processed_signal:
        errors[i] = {}
        for j in processed_signal[i]:
            signal_dropna = processed_signal[i][j].drop(
                processed_signal[i][j][processed_signal[i][j]['lead'].isna()].index)
            if fit_method == 'mean':
                error = signal_dropna['lead'].mean()
            elif fit_method == 'abs':
                error = abs(signal_dropna['lead']).mean()
            elif fit_method == 'rmse':
                mse = np.square(np.subtract(np.zeros_like(signal_dropna['lead']),signal_dropna['lead'])).mean() 
                rmse = math.sqrt(mse)
                error = rmse
            errors[i][j] = error
    return errors


In [15]:
def get_best_fit_harm_function(processed_signal, errors):
    best_error = {}
    best_fit_harm = {}
    for i in processed_signal:
        best_error[i] = pd.Series(errors[i]).abs().min()
        best_fit_harm[i] = pd.Series(errors[i]).abs().idxmin()
    return best_fit_harm, best_error


In [16]:
def find_pv_lead_function_2(data, processed_signal):
    for d in data:
        for p in processed_signal[d]:
            # processed_signal[d][p]['pv'] = pd.Series(dtype='str')
            processing_signal = processed_signal[d][p].tail(len(data[d]))
            p_data = pd.DataFrame(
                {'peaks': data[d]['peaks'], 'count': range(len(data[d]))})
            p_data = p_data.drop(p_data[p_data['peaks'].isna()].index)
            p_data_count = list(p_data['count'])
            p_signal = pd.DataFrame(
                {'peaks': processing_signal['peaks'], 'count': range(len(processing_signal))})
            p_signal = p_signal.drop(p_signal[p_signal['peaks'].isna()].index)
            p_signal_list = list(p_signal['count'])
            p_lead = []
            for i in range(0, len(p_signal_list)):
                temp = []
                temp_abs = []
                temp_2 = []
                for j in range(0, len(p_data_count)):
                    temp.append((p_data_count[j] - p_signal_list[i]))
                    temp_abs.append(abs(p_data_count[j] - p_signal_list[i]))
                for k in range(0, len(temp_abs)):
                    if temp_abs[k] == min(temp_abs):
                        temp_2 = temp[k]
                p_lead.append(temp_2)
            p_signal['lead'] = p_lead

            v_data = pd.DataFrame(
                {'valleys': data[d]['valleys'], 'count': range(len(data[d]))})
            v_data = v_data.drop(v_data[v_data['valleys'].isna()].index)
            v_data_count = list(v_data['count'])
            v_signal = pd.DataFrame(
                {'valleys': processing_signal['valleys'], 'count': range(len(processing_signal))})
            v_signal = v_signal.drop(
                v_signal[v_signal['valleys'].isna()].index)
            v_signal_list = list(v_signal['count'])
            v_lead = []
            for i in range(0, len(v_signal_list)):
                temp = []
                temp_abs = []
                temp_2 = []
                for j in range(0, len(v_data_count)):
                    temp.append((v_data_count[j] - v_signal_list[i]))
                    temp_abs.append(abs(v_data_count[j] - v_signal_list[i]))
                for k in range(0, len(temp_abs)):
                    if temp_abs[k] == min(temp_abs):
                        temp_2 = temp[k]
                v_lead.append(temp_2)
            v_signal['lead'] = v_lead

            processed_signal[d][p]['lead'] = pd.Series(dtype='float64')
            processed_signal[d][p]['lead'].loc[p_signal['lead'].index] = p_signal['lead']
            # processed_signal[d][p]['pv'].loc[p_signal['lead'].index] = 'peak'
            processed_signal[d][p]['lead'].loc[v_signal['lead'].index] = v_signal['lead']
            # processed_signal[d][p]['pv'].loc[v_signal['lead'].index] = 'valley'


In [17]:
def get_first_lead_function(processed_signal, best_fit_harm):
    first_date = {}
    lead = {}
    pv = {}
    for i in processed_signal:
        harm = best_fit_harm[i]
        temp = processed_signal[i][harm].loc[list(
            processed_signal[i][harm]['lead'].dropna().index)[0]]
        first_date[i] = list(processed_signal[i][harm]
                             ['lead'].dropna().index)[0]
        lead[i] = temp['lead']
        pv[i] = temp['pv']
    return first_date, lead, pv


In [18]:
def select_model(test_data, processed_signal, fit_method):
    errors = get_fit_error_function(processed_signal, fit_method)
    best_fit_harm, best_error = get_best_fit_harm_function(
        processed_signal, errors)
    find_pv_lead_function_2(test_data, processed_signal)
    first_date, lead, pv = get_first_lead_function(
        processed_signal, best_fit_harm)
    return errors, best_fit_harm, best_error, first_date, lead, pv
    # return errors, best_fit_harm, best_error


# Evaluate Model

In [19]:
def built_result_table_function(processed_signal, test_data_start_list, lead, pv, best_error, best_fit_harm):
    result_table = pd.DataFrame(columns=[
        's_date', 't_date', 'lead', 'ans_date', 'pv', 'error', 'best_fit'])
    for i in processed_signal:
        result_table.loc[i, 'error'] = round(best_error[i], 2)
        result_table.loc[i, 'best_fit'] = best_fit_harm[i]
        result_table.loc[i, 'lead'] = lead[i]
        result_table.loc[i, 'pv'] = pv[i]
    result_table['s_date'] = test_data_start_list
    return result_table


In [20]:
def result_table_process_function(result_table, all_data, first_date, data_range):
    for i in result_table.index:
        t_date = all_data.iloc[all_data.index.get_loc(result_table.loc[i, 's_date']) + first_date[i] - data_range].name
        t_date = datetime.datetime.strftime(t_date, '%Y-%m-%d')
        result_table.loc[i, 't_date'] = t_date
        ans = all_data.iloc[int(all_data.index.get_loc(result_table.loc[i, 't_date']) + result_table.loc[i, 'lead'])].name
        ans = datetime.datetime.strftime(ans, '%Y-%m-%d')
        result_table.loc[i, 'ans_date'] = ans


In [21]:
def compute_final_error_function(result_table):
    final_error = round(
        sum([abs(ele) for ele in result_table['lead']]) / len(result_table['lead']), 2)
    return final_error


In [22]:
def evaluate_model(processed_signal, test_data_start_list, all_data, best_fit_harm, best_error, first_date, lead, pv, data_range):
    result_table = built_result_table_function(
        processed_signal, test_data_start_list, lead, pv, best_error, best_fit_harm)
    result_table_process_function(result_table, all_data, first_date, data_range)
    final_error = compute_final_error_function(result_table)
    return result_table, final_error


# Main function

In [23]:
def main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
        n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method):

    # 1. Load data
    train_data, test_data, all_data, test_data_start_list = load_data(
    stock_name, date_predict_start, data_range, slide_range, n_slide)
    # 2. Preprocessing
    preprocessing(train_data, test_data, pv_range, pv_method)
    # 3. Build model
    harmonics, model = build_model(
        train_data, n_harm_lower_limit, n_harm_upper_limit, pv_range, data_range)
    # 4. Select model
    errors, best_fit_harm, best_error, first_date, lead, pv = select_model(test_data, 
        model, fit_method)
    # 5. Evaluate model
    result_table, final_error = evaluate_model(
        model, test_data_start_list, all_data, best_fit_harm, best_error, first_date, lead, pv, data_range)
    print('final_error = ', final_error)
    print(result_table)
    
    return harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error


In [108]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  2.0
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  0.89       37
data_1  2021-01-19  2021-01-20 -1.0  2021-01-19  valley  0.84       27
data_2  2021-02-02  2021-02-02  6.0  2021-02-10    peak  0.74       31
data_3  2021-02-17  2021-02-19  2.0  2021-02-23  valley  0.76       31
data_4  2021-03-03  2021-03-05 -1.0  2021-03-04  valley  0.77       28
data_5  2021-03-17  2021-03-17  2.0  2021-03-19  valley   0.7       32
data_6  2021-03-31  2021-04-01 -1.0  2021-03-31  valley  0.79       31
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  0.85       37
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley  0.88       33
data_9  2021-05-13  2021-05-13  4.0  2021-05-19  valley  0.78       31
data_10 2021-05-27  2021-06-02 -1.0  2021-06-01    peak   0.8       36
data_11 2021-06-11  2021-06-17 -2.0  2021-06-15    peak  0.82       33
data_12 2021-06-25  2021-06-30  4.0  2021-07-07    peak   

In [109]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.65
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  0.86       28
data_1  2021-01-19  2021-01-20 -1.0  2021-01-19  valley  0.87       39
data_2  2021-02-02  2021-02-02  4.0  2021-02-08    peak  0.82       40
data_3  2021-02-17  2021-02-19  1.0  2021-02-22  valley  0.84       39
data_4  2021-03-03  2021-03-04  0.0  2021-03-04  valley  0.79       38
data_5  2021-03-17  2021-03-18  1.0  2021-03-19  valley  0.76       39
data_6  2021-03-31  2021-04-07  2.0  2021-04-09    peak  0.73       38
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  0.72       38
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley  0.59       37
data_9  2021-05-13  2021-05-13  0.0  2021-05-13  valley  0.54       31
data_10 2021-05-27  2021-06-02  0.0  2021-06-02    peak  0.63       32
data_11 2021-06-11  2021-06-17  3.0  2021-06-22    peak  0.57       26
data_12 2021-06-25  2021-06-30  2.0  2021-07-02    peak  

In [110]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.77
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  0.04       35
data_1  2021-01-19  2021-01-21  2.0  2021-01-25  valley  0.08       25
data_2  2021-02-02  2021-02-05  3.0  2021-02-10  valley  0.02       33
data_3  2021-02-17  2021-02-19  2.0  2021-02-23  valley   0.0       33
data_4  2021-03-03  2021-03-10  1.0  2021-03-11    peak  0.07       20
data_5  2021-03-17  2021-03-17  2.0  2021-03-19  valley   0.0       32
data_6  2021-03-31  2021-04-01 -1.0  2021-03-31  valley  0.07       35
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  0.08       35
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley  0.02       31
data_9  2021-05-13  2021-05-13  4.0  2021-05-19  valley  0.03       31
data_10 2021-05-27  2021-06-02 -1.0  2021-06-01    peak  0.03       25
data_11 2021-06-11  2021-06-17 -2.0  2021-06-15    peak  0.02       33
data_12 2021-06-25  2021-06-30  4.0  2021-07-07    peak  

In [111]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.81
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley   0.0       22
data_1  2021-01-19  2021-01-20 -1.0  2021-01-19  valley   0.0       40
data_2  2021-02-02  2021-02-04 -2.0  2021-02-02  valley   0.0       23
data_3  2021-02-17  2021-02-19  1.0  2021-02-22  valley   0.0       39
data_4  2021-03-03  2021-03-04  0.0  2021-03-04  valley  0.07       36
data_5  2021-03-17  2021-03-17  2.0  2021-03-19  valley  0.11       24
data_6  2021-03-31  2021-03-31  2.0  2021-04-05    peak  0.07       39
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  0.07       40
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley   0.0       34
data_9  2021-05-13  2021-05-13  0.0  2021-05-13  valley   0.0       28
data_10 2021-05-27  2021-06-02  0.0  2021-06-02    peak  0.02       35
data_11 2021-06-11  2021-06-17  3.0  2021-06-22    peak  0.02       31
data_12 2021-06-25  2021-06-30  2.0  2021-07-02    peak  

In [112]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  2.0
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  1.25       30
data_1  2021-01-19  2021-01-20 -1.0  2021-01-19  valley  1.08       27
data_2  2021-02-02  2021-02-05  3.0  2021-02-10  valley  1.01       27
data_3  2021-02-17  2021-02-19  2.0  2021-02-23  valley  1.04       27
data_4  2021-03-03  2021-03-04  0.0  2021-03-04  valley  1.04       35
data_5  2021-03-17  2021-03-17  2.0  2021-03-19  valley  0.96       31
data_6  2021-03-31  2021-04-01 -1.0  2021-03-31  valley   1.0       31
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  1.14       37
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley  1.13       33
data_9  2021-05-13  2021-05-13  4.0  2021-05-19  valley  0.97       31
data_10 2021-05-27  2021-06-02 -1.0  2021-06-01    peak  1.04       31
data_11 2021-06-11  2021-06-18 -3.0  2021-06-15    peak  1.08       36
data_12 2021-06-25  2021-06-30  4.0  2021-07-07    peak  1

In [113]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.85
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  1.14       28
data_1  2021-01-19  2021-01-20 -1.0  2021-01-19  valley  1.56       28
data_2  2021-02-02  2021-02-05  3.0  2021-02-10  valley  1.27       28
data_3  2021-02-17  2021-02-19  1.0  2021-02-22  valley  1.41       39
data_4  2021-03-03  2021-03-04  0.0  2021-03-04  valley   1.4       37
data_5  2021-03-17  2021-03-18  1.0  2021-03-19  valley  1.37       37
data_6  2021-03-31  2021-04-07  2.0  2021-04-09    peak  1.48       38
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  1.33       38
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley  0.85       39
data_9  2021-05-13  2021-05-13  0.0  2021-05-13  valley  0.81       31
data_10 2021-05-27  2021-06-02  0.0  2021-06-02    peak  0.88       39
data_11 2021-06-11  2021-06-18  2.0  2021-06-22    peak  0.83       40
data_12 2021-06-25  2021-06-30  2.0  2021-07-02    peak  

In [114]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 5
n_slide = 52
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.98
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  0.89       37
data_1  2021-01-11  2021-01-15   0.0  2021-01-15  valley  0.86       27
data_2  2021-01-19  2021-01-20  -1.0  2021-01-19  valley  0.84       27
data_3  2021-01-26  2021-01-28   1.0  2021-01-29  valley  0.76       28
data_4  2021-02-02  2021-02-02   6.0  2021-02-10    peak  0.74       31
data_5  2021-02-09  2021-02-10   0.0  2021-02-10  valley  0.76       31
data_6  2021-02-17  2021-02-19   2.0  2021-02-23  valley  0.76       31
data_7  2021-02-24  2021-03-02   2.0  2021-03-04  valley  0.77       27
data_8  2021-03-03  2021-03-05  -1.0  2021-03-04  valley  0.77       28
data_9  2021-03-10  2021-03-15   2.0  2021-03-17    peak  0.72       29
data_10 2021-03-17  2021-03-17   2.0  2021-03-19  valley   0.7       32
data_11 2021-03-24  2021-03-25   2.0  2021-03-29    peak  0.73       28
data_12 2021-03-31  2021-04-01  -1.0  2021-0

In [115]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 5
n_slide = 52
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.48
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  0.86       28
data_1  2021-01-11  2021-01-19 -1.0  2021-01-15  valley  0.91       39
data_2  2021-01-19  2021-01-20 -1.0  2021-01-19  valley  0.87       39
data_3  2021-01-26  2021-01-28  1.0  2021-01-29  valley  0.78       39
data_4  2021-02-02  2021-02-02  4.0  2021-02-08    peak  0.82       40
data_5  2021-02-09  2021-02-10  0.0  2021-02-10  valley  0.82       39
data_6  2021-02-17  2021-02-19  1.0  2021-02-22  valley  0.84       39
data_7  2021-02-24  2021-03-01 -1.0  2021-02-26  valley  0.77       39
data_8  2021-03-03  2021-03-04  0.0  2021-03-04  valley  0.79       38
data_9  2021-03-10  2021-03-17  0.0  2021-03-17    peak  0.73       38
data_10 2021-03-17  2021-03-18  1.0  2021-03-19  valley  0.76       39
data_11 2021-03-24  2021-03-25  1.0  2021-03-26    peak  0.76       39
data_12 2021-03-31  2021-04-07  2.0  2021-04-09    peak  

In [116]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 5
n_slide = 52
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.88
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  0.04       35
data_1  2021-01-11  2021-01-15   0.0  2021-01-15  valley   0.2       32
data_2  2021-01-19  2021-01-21   2.0  2021-01-25  valley  0.08       25
data_3  2021-01-26  2021-01-28   1.0  2021-01-29  valley   0.0       34
data_4  2021-02-02  2021-02-05   3.0  2021-02-10  valley  0.02       33
data_5  2021-02-09  2021-02-10   0.0  2021-02-10  valley  0.02       34
data_6  2021-02-17  2021-02-19   2.0  2021-02-23  valley   0.0       33
data_7  2021-02-24  2021-03-02   2.0  2021-03-04  valley  0.02       35
data_8  2021-03-03  2021-03-10   1.0  2021-03-11    peak  0.07       20
data_9  2021-03-10  2021-03-15   2.0  2021-03-17    peak  0.06       29
data_10 2021-03-17  2021-03-17   2.0  2021-03-19  valley   0.0       32
data_11 2021-03-24  2021-03-25   2.0  2021-03-29    peak  0.11       29
data_12 2021-03-31  2021-04-01  -1.0  2021-0

In [117]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 5
n_slide = 52
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.85
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley   0.0       22
data_1  2021-01-11  2021-01-19 -1.0  2021-01-15  valley   0.0       22
data_2  2021-01-19  2021-01-20 -1.0  2021-01-19  valley   0.0       40
data_3  2021-01-26  2021-02-04  4.0  2021-02-10  valley  0.05       23
data_4  2021-02-02  2021-02-04 -2.0  2021-02-02  valley   0.0       23
data_5  2021-02-09  2021-02-17  3.0  2021-02-22  valley  0.05       25
data_6  2021-02-17  2021-02-19  1.0  2021-02-22  valley   0.0       39
data_7  2021-02-24  2021-03-01 -1.0  2021-02-26  valley  0.09       39
data_8  2021-03-03  2021-03-04  0.0  2021-03-04  valley  0.07       36
data_9  2021-03-10  2021-03-10  3.0  2021-03-15    peak  0.07       39
data_10 2021-03-17  2021-03-17  2.0  2021-03-19  valley  0.11       24
data_11 2021-03-24  2021-04-01 -2.0  2021-03-30  valley  0.03       20
data_12 2021-03-31  2021-03-31  2.0  2021-04-05    peak  

In [118]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 5
n_slide = 52
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.96
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  1.25       30
data_1  2021-01-11  2021-01-15   0.0  2021-01-15  valley  1.16       28
data_2  2021-01-19  2021-01-20  -1.0  2021-01-19  valley  1.08       27
data_3  2021-01-26  2021-01-28   1.0  2021-01-29  valley  1.02       28
data_4  2021-02-02  2021-02-05   3.0  2021-02-10  valley  1.01       27
data_5  2021-02-09  2021-02-10   0.0  2021-02-10  valley  1.07       31
data_6  2021-02-17  2021-02-19   2.0  2021-02-23  valley  1.04       27
data_7  2021-02-24  2021-03-01  -1.0  2021-02-26  valley  1.05       31
data_8  2021-03-03  2021-03-04   0.0  2021-03-04  valley  1.04       35
data_9  2021-03-10  2021-03-24   3.0  2021-03-29    peak  1.01       35
data_10 2021-03-17  2021-03-17   2.0  2021-03-19  valley  0.96       31
data_11 2021-03-24  2021-03-25   2.0  2021-03-29    peak   1.0       28
data_12 2021-03-31  2021-04-01  -1.0  2021-0

In [119]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 5
n_slide = 52
pv_range = 2
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.5
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  1.14       28
data_1  2021-01-11  2021-01-15  0.0  2021-01-15  valley  1.64       28
data_2  2021-01-19  2021-01-20 -1.0  2021-01-19  valley  1.56       28
data_3  2021-01-26  2021-01-28  1.0  2021-01-29  valley  1.37       39
data_4  2021-02-02  2021-02-05  3.0  2021-02-10  valley  1.27       28
data_5  2021-02-09  2021-02-10  0.0  2021-02-10  valley  1.38       39
data_6  2021-02-17  2021-02-19  1.0  2021-02-22  valley  1.41       39
data_7  2021-02-24  2021-03-01 -1.0  2021-02-26  valley  1.37       39
data_8  2021-03-03  2021-03-04  0.0  2021-03-04  valley   1.4       37
data_9  2021-03-10  2021-03-17  0.0  2021-03-17    peak  1.35       38
data_10 2021-03-17  2021-03-18  1.0  2021-03-19  valley  1.37       37
data_11 2021-03-24  2021-03-25  1.0  2021-03-26    peak  1.33       39
data_12 2021-03-31  2021-04-07  2.0  2021-04-09    peak  1

In [120]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 5
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  7.0
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  3.13       32
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  2.66       34
data_2  2021-02-02  2021-02-05  -3.0  2021-02-02  valley  2.45       28
data_3  2021-02-17  2021-02-19   9.0  2021-03-04  valley  2.96       29
data_4  2021-03-03  2021-03-04   0.0  2021-03-04  valley   2.5       36
data_5  2021-03-17  2021-03-24  16.0  2021-04-16    peak  2.33       27
data_6  2021-03-31  2021-04-07   7.0  2021-04-16    peak  2.46       39
data_7  2021-04-15  2021-04-16   2.0  2021-04-20  valley  2.33       39
data_8  2021-04-29  2021-05-03   7.0  2021-05-12  valley  2.21       32
data_9  2021-05-13  2021-05-13   4.0  2021-05-19  valley  2.25       31
data_10 2021-05-27  2021-06-01  10.0  2021-06-15    peak   1.7       40
data_11 2021-06-11  2021-06-17  18.0  2021-07-14    peak  1.89       31
data_12 2021-06-25  2021-07-01   8.0  2021-07

In [121]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 5
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  6.31
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  2.07       30
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley   2.0       39
data_2  2021-02-02  2021-02-05  -3.0  2021-02-02  valley   2.0       39
data_3  2021-02-17  2021-02-19   9.0  2021-03-04  valley  1.93       39
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley  2.04       39
data_5  2021-03-17  2021-03-18   4.0  2021-03-24  valley  1.96       39
data_6  2021-03-31  2021-04-07   7.0  2021-04-16    peak  1.93       39
data_7  2021-04-15  2021-04-16   2.0  2021-04-20  valley  1.78       38
data_8  2021-04-29  2021-05-03   7.0  2021-05-12  valley  2.15       38
data_9  2021-05-13  2021-05-13   0.0  2021-05-13  valley   2.0       31
data_10 2021-05-27  2021-06-01   9.0  2021-06-14    peak  1.26       29
data_11 2021-06-11  2021-06-17  16.0  2021-07-12    peak  1.54       31
data_12 2021-06-25  2021-07-01   6.0  2021-0

In [122]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 5
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  6.12
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  0.06       26
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley   0.0       31
data_2  2021-02-02  2021-02-05  -3.0  2021-02-02  valley   0.0       37
data_3  2021-02-17  2021-02-19   9.0  2021-03-04  valley  0.38       31
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley  0.07       26
data_5  2021-03-17  2021-03-18   5.0  2021-03-25  valley  0.03       37
data_6  2021-03-31  2021-04-06   8.0  2021-04-16    peak  0.11       27
data_7  2021-04-15  2021-04-19   1.0  2021-04-20  valley  0.46       23
data_8  2021-04-29  2021-05-03   7.0  2021-05-12  valley  0.04       29
data_9  2021-05-13  2021-06-01   0.0  2021-06-01    peak  0.52       20
data_10 2021-05-27  2021-06-03   8.0  2021-06-15    peak  0.18       21
data_11 2021-06-11  2021-06-18  17.0  2021-07-14    peak  0.11       38
data_12 2021-06-25  2021-06-30   9.0  2021-0

In [123]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 5
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  6.31
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   0.0       39
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  0.06       40
data_2  2021-02-02  2021-02-04  -2.0  2021-02-02  valley  0.04       25
data_3  2021-02-17  2021-02-22   8.0  2021-03-04  valley  0.21       26
data_4  2021-03-03  2021-03-04   0.0  2021-03-04  valley  0.41       34
data_5  2021-03-17  2021-03-18   4.0  2021-03-24  valley  0.24       37
data_6  2021-03-31  2021-04-05   9.0  2021-04-16    peak  0.43       26
data_7  2021-04-15  2021-04-19   1.0  2021-04-20  valley  0.04       26
data_8  2021-04-29  2021-05-04   6.0  2021-05-12  valley   0.0       22
data_9  2021-05-13  2021-05-13   0.0  2021-05-13  valley   1.0       23
data_10 2021-05-27  2021-06-02   8.0  2021-06-14    peak  0.12       20
data_11 2021-06-11  2021-06-17  16.0  2021-07-12    peak  0.48       22
data_12 2021-06-25  2021-06-30   7.0  2021-0

In [124]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 5
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  7.0
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  5.21       28
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley  4.53       24
data_2  2021-02-02  2021-02-05  -3.0  2021-02-02  valley  4.25       28
data_3  2021-02-17  2021-02-19   9.0  2021-03-04  valley  5.58       29
data_4  2021-03-03  2021-03-04   0.0  2021-03-04  valley  4.46       36
data_5  2021-03-17  2021-03-24  16.0  2021-04-16    peak  4.29       27
data_6  2021-03-31  2021-04-01  10.0  2021-04-16    peak  4.37       22
data_7  2021-04-15  2021-04-16   2.0  2021-04-20  valley  4.09       39
data_8  2021-04-29  2021-05-03   7.0  2021-05-12  valley  4.06       32
data_9  2021-05-13  2021-05-13   4.0  2021-05-19  valley  4.01       21
data_10 2021-05-27  2021-06-02   9.0  2021-06-15    peak  3.22       31
data_11 2021-06-11  2021-06-17  18.0  2021-07-14    peak  3.11       22
data_12 2021-06-25  2021-07-01   8.0  2021-07

In [125]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 5
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  6.62
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   3.9       32
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley  3.93       22
data_2  2021-02-02  2021-02-05  -3.0  2021-02-02  valley  3.85       26
data_3  2021-02-17  2021-02-19   9.0  2021-03-04  valley  3.84       29
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley  4.08       26
data_5  2021-03-17  2021-03-24  16.0  2021-04-16    peak  3.62       26
data_6  2021-03-31  2021-04-07   7.0  2021-04-16    peak  3.97       39
data_7  2021-04-15  2021-04-19   1.0  2021-04-20  valley  3.28       23
data_8  2021-04-29  2021-05-03   7.0  2021-05-12  valley  4.21       25
data_9  2021-05-13  2021-05-13   0.0  2021-05-13  valley  4.11       32
data_10 2021-05-27  2021-06-01   9.0  2021-06-14    peak  2.28       29
data_11 2021-06-11  2021-06-17  16.0  2021-07-12    peak  2.64       21
data_12 2021-06-25  2021-06-30   7.0  2021-0

In [126]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 10
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  16.69
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  7.13       27
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley   6.5       24
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley  7.33       27
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley  7.69       26
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley  8.08       27
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  7.31       25
data_6  2021-03-31  2021-04-07  22.0  2021-05-07    peak   8.0       29
data_7  2021-04-15  2021-04-20  16.0  2021-05-12  valley  5.94       22
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley   5.6       20
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak  5.25       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak   5.6       25
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak   6.5       20
data_12 2021-06-25  2021-06-30  45.0  2021-

In [127]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 10
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  15.58
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   6.36       31
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley   7.21       27
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley   6.83       27
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley    8.0       27
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley    8.0       27
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak   7.38       25
data_6  2021-03-31  2021-04-07  22.0  2021-05-07    peak    8.5       29
data_7  2021-04-15  2021-04-19  17.0  2021-05-12  valley    9.2       26
data_8  2021-04-29  2021-05-13  -1.0  2021-05-12  valley   8.57       26
data_9  2021-05-13  2021-05-13   0.0  2021-05-13  valley   8.73       32
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak    8.8       32
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak   9.07       24
data_12 2021-06-25  2021-06-30

In [128]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 10
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  14.85
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  0.12       20
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley   4.0       26
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley   2.5       27
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley  0.92       26
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley  1.31       27
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  1.31       27
data_6  2021-03-31  2021-04-07  22.0  2021-05-07    peak   0.0       35
data_7  2021-04-15  2021-04-20  16.0  2021-05-12  valley  0.31       22
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley  0.38       37
data_9  2021-05-13  2021-06-08   8.0  2021-06-18  valley   0.0       35
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak  0.81       34
data_11 2021-06-11  2021-06-18  53.0  2021-09-02    peak  0.06       34
data_12 2021-06-25  2021-07-01  44.0  2021-

In [129]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 10
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  15.81
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  0.19       20
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  0.14       26
data_2  2021-02-02  2021-02-02  21.0  2021-03-04  valley   0.2       20
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley  1.54       27
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley  1.69       27
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  1.69       27
data_6  2021-03-31  2021-04-07  22.0  2021-05-07    peak  0.12       31
data_7  2021-04-15  2021-04-16  18.0  2021-05-12  valley  0.33       40
data_8  2021-04-29  2021-05-13  -1.0  2021-05-12  valley   1.5       21
data_9  2021-05-13  2021-05-13   0.0  2021-05-13  valley  0.38       27
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak  0.54       27
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  1.33       23
data_12 2021-06-25  2021-06-30  45.0  2021-

In [130]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 10
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  16.96
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  12.12       27
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley   10.8       24
data_2  2021-02-02  2021-02-04  19.0  2021-03-04  valley  12.14       23
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley  11.94       26
data_4  2021-03-03  2021-03-04   0.0  2021-03-04  valley  12.36       23
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  12.07       25
data_6  2021-03-31  2021-04-05  24.0  2021-05-07    peak  13.22       26
data_7  2021-04-15  2021-04-20  16.0  2021-05-12  valley    9.8       22
data_8  2021-04-29  2021-05-13  -1.0  2021-05-12  valley   8.67       25
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak   8.94       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak   8.97       25
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak   9.95       20
data_12 2021-06-25  2021-06-30

In [131]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 10
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  15.77
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  11.32       31
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley  12.67       25
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley  13.06       27
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley   14.6       27
data_4  2021-03-03  2021-03-05  -1.0  2021-03-04  valley   14.6       27
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  13.59       25
data_6  2021-03-31  2021-04-07  22.0  2021-05-07    peak  14.92       29
data_7  2021-04-15  2021-04-20  16.0  2021-05-12  valley  14.76       22
data_8  2021-04-29  2021-05-14  -2.0  2021-05-12  valley  13.71       24
data_9  2021-05-13  2021-05-13   0.0  2021-05-13  valley  13.92       24
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak  14.06       24
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  13.94       24
data_12 2021-06-25  2021-06-30

In [132]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 15
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  20.04
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   7.7       33
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  7.36       29
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley   6.0       30
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley   6.0       30
data_4  2021-03-03  2021-03-23  32.0  2021-05-07    peak   6.5       20
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  9.91       33
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley  4.64       29
data_7  2021-04-15  2021-04-19  17.0  2021-05-12  valley   4.3       33
data_8  2021-04-29  2021-05-14  -2.0  2021-05-12  valley  5.45       23
data_9  2021-05-13  2021-06-18  53.0  2021-09-02    peak  4.83       34
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak  5.18       34
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak   5.6       31
data_12 2021-06-25  2021-06-30  45.0  2021-

In [133]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 15
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  20.23
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   14.3       38
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  12.55       32
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley   11.0       30
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley   8.22       30
data_4  2021-03-03  2021-03-23  32.0  2021-05-07    peak   8.17       20
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak   6.27       33
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley   6.18       29
data_7  2021-04-15  2021-04-19  17.0  2021-05-12  valley    6.0       25
data_8  2021-04-29  2021-05-14  -2.0  2021-05-12  valley   6.91       23
data_9  2021-05-13  2021-06-18  53.0  2021-09-02    peak   6.17       34
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak   6.82       34
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak   7.36       25
data_12 2021-06-25  2021-06-30

In [134]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 15
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  20.15
            s_date      t_date  lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   0.4       23
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley  0.73       24
data_2  2021-02-02  2021-02-02  21.0  2021-03-04  valley   1.0       20
data_3  2021-02-17  2021-03-01   3.0  2021-03-04  valley  0.45       40
data_4  2021-03-03  2021-03-08  -2.0  2021-03-04  valley  0.45       32
data_5  2021-03-17  2021-03-23  32.0  2021-05-07    peak   2.5       21
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley   0.0       35
data_7  2021-04-15  2021-04-19  17.0  2021-05-12  valley   0.1       33
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley  0.09       20
data_9  2021-05-13  2021-06-18  53.0  2021-09-02    peak  1.33       34
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak   1.1       31
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  0.09       20
data_12 2021-06-25  2021-06-30  45.0  2021-

In [135]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 15
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  19.58
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley    6.7       35
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley   0.33       21
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley    0.3       40
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley   0.27       22
data_4  2021-03-03  2021-03-03   1.0  2021-03-04  valley   0.25       22
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak   0.83       39
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley   0.36       23
data_7  2021-04-15  2021-04-19  17.0  2021-05-12  valley    0.0       29
data_8  2021-04-29  2021-05-13  -1.0  2021-05-12  valley   0.08       26
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak   0.25       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak    0.0       21
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak    0.0       22
data_12 2021-06-25  2021-06-30

In [136]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 15
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  20.0
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  13.86       33
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  13.16       29
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley  11.87       30
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley  11.87       30
data_4  2021-03-03  2021-03-23  32.0  2021-05-07    peak  11.65       20
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  20.05       33
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley   8.03       29
data_7  2021-04-15  2021-04-19  17.0  2021-05-12  valley   7.89       25
data_8  2021-04-29  2021-05-14  -2.0  2021-05-12  valley   8.62       23
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak   9.04       20
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak    9.2       31
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak   9.22       31
data_12 2021-06-25  2021-06-30 

In [137]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 15
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  19.96
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  24.47       38
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  21.95       29
data_2  2021-02-02  2021-02-05  18.0  2021-03-04  valley  19.32       29
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley  16.24       24
data_4  2021-03-03  2021-03-23  32.0  2021-05-07    peak  15.23       20
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  12.88       23
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley  12.62       23
data_7  2021-04-15  2021-04-19  17.0  2021-05-12  valley  12.62       25
data_8  2021-04-29  2021-05-14  -2.0  2021-05-12  valley  13.05       24
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak  13.33       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak  13.52       25
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  13.38       24
data_12 2021-06-25  2021-06-30

In [138]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 20
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  20.54
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  11.33       32
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley   9.78       26
data_2  2021-02-02  2021-03-01   3.0  2021-03-04  valley  11.22       39
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley   8.22       30
data_4  2021-03-03  2021-03-03   1.0  2021-03-04  valley   17.6       22
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak   15.4       25
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley   9.75       20
data_7  2021-04-15  2021-06-16  55.0  2021-09-02    peak  10.56       20
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley  12.44       20
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak   13.0       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak  13.44       22
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  13.44       22
data_12 2021-06-25  2021-07-12

In [139]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 20
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'abs'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  22.92
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  11.33       32
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley   9.44       26
data_2  2021-02-02  2021-03-01   3.0  2021-03-04  valley  10.89       39
data_3  2021-02-17  2021-03-02   2.0  2021-03-04  valley   8.22       30
data_4  2021-03-03  2021-03-03   1.0  2021-03-04  valley   11.3       22
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak   15.4       25
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley   9.62       20
data_7  2021-04-15  2021-06-16  55.0  2021-09-02    peak  10.44       20
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley  12.11       20
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak  13.11       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak  13.33       22
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  13.33       22
data_12 2021-06-25  2021-07-12

In [140]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 20
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  22.12
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   3.11       32
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley   3.78       23
data_2  2021-02-02  2021-03-03   1.0  2021-03-04  valley   2.44       24
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley   6.56       24
data_4  2021-03-03  2021-03-03   1.0  2021-03-04  valley    4.0       22
data_5  2021-03-17  2021-03-23  32.0  2021-05-07    peak    0.0       21
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley    0.0       26
data_7  2021-04-15  2021-06-18  53.0  2021-09-02    peak   0.36       32
data_8  2021-04-29  2021-05-13  -1.0  2021-05-12  valley   0.64       21
data_9  2021-05-13  2021-06-18  53.0  2021-09-02    peak   1.18       34
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak   1.09       34
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak   2.11       20
data_12 2021-06-25  2021-06-30

In [141]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 20
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'mean'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  25.62
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   2.89       32
data_1  2021-01-19  2021-01-21   6.0  2021-01-29  valley   3.67       23
data_2  2021-02-02  2021-03-03   1.0  2021-03-04  valley   2.78       24
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley    7.0       24
data_4  2021-03-03  2021-03-03   1.0  2021-03-04  valley   10.3       22
data_5  2021-03-17  2021-03-23  32.0  2021-05-07    peak    0.3       22
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley   0.88       20
data_7  2021-04-15  2021-06-18  53.0  2021-09-02    peak   0.09       32
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley    0.2       34
data_9  2021-05-13  2021-06-18  53.0  2021-09-02    peak   1.09       34
data_10 2021-05-27  2021-06-18  53.0  2021-09-02    peak    1.0       34
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak    2.0       20
data_12 2021-06-25  2021-06-30

In [142]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 20
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  21.96
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley  24.08       32
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  21.13       26
data_2  2021-02-02  2021-03-01   3.0  2021-03-04  valley  19.89       39
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley   15.7       24
data_4  2021-03-03  2021-03-03   1.0  2021-03-04  valley  27.79       22
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  24.71       25
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley  16.67       26
data_7  2021-04-15  2021-06-17  54.0  2021-09-02    peak  18.96       26
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley  22.26       20
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak  24.33       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak  24.29       24
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  24.19       24
data_12 2021-06-25  2021-06-30

In [143]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 20
n_harm_lower_limit = 20
n_harm_upper_limit = 40
fit_method = 'rmse'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  24.38
            s_date      t_date  lead    ans_date      pv  error best_fit
data_0  2021-01-04  2021-01-06  -2.0  2021-01-04  valley   23.8       32
data_1  2021-01-19  2021-01-20   7.0  2021-01-29  valley  20.81       26
data_2  2021-02-02  2021-03-01   3.0  2021-03-04  valley   19.7       39
data_3  2021-02-17  2021-03-03   1.0  2021-03-04  valley   15.9       24
data_4  2021-03-03  2021-03-03   1.0  2021-03-04  valley  19.69       22
data_5  2021-03-17  2021-03-24  31.0  2021-05-07    peak  24.71       25
data_6  2021-03-31  2021-04-14  -9.0  2021-03-31  valley  17.52       26
data_7  2021-04-15  2021-06-17  54.0  2021-09-02    peak  18.73       26
data_8  2021-04-29  2021-05-12   0.0  2021-05-12  valley  21.69       20
data_9  2021-05-13  2021-06-16  55.0  2021-09-02    peak  24.33       20
data_10 2021-05-27  2021-06-17  54.0  2021-09-02    peak  24.29       24
data_11 2021-06-11  2021-06-17  54.0  2021-09-02    peak  24.19       24
data_12 2021-06-25  2021-06-30

In [144]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 1
n_harm_upper_limit = 70
fit_method = 'abs'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  2.0
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-05 -1.0  2021-01-04  valley  0.85       70
data_1  2021-01-19  2021-05-10 -1.0  2021-05-07    peak  0.33        1
data_2  2021-02-02  2021-02-02  6.0  2021-02-10    peak  0.74       31
data_3  2021-02-17  2021-02-19  2.0  2021-02-23  valley  0.76       31
data_4  2021-03-03  2021-03-05 -1.0  2021-03-04  valley  0.77       28
data_5  2021-03-17  2021-03-22 -3.0  2021-03-17    peak  0.33        1
data_6  2021-03-31  2021-04-01 -1.0  2021-03-31  valley  0.79       31
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  0.85       37
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley  0.88       33
data_9  2021-05-13  2021-05-13  4.0  2021-05-19  valley  0.78       31
data_10 2021-05-27  2021-05-27  2.0  2021-06-01    peak  0.75        3
data_11 2021-06-11  2021-06-17 -2.0  2021-06-15    peak  0.82       33
data_12 2021-06-25  2021-06-30  4.0  2021-07-07    peak   

In [145]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 1
n_harm_upper_limit = 70
fit_method = 'abs'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.96
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  0.86       28
data_1  2021-01-19  2021-05-10 -1.0  2021-05-07    peak  0.67        1
data_2  2021-02-02  2021-02-02  4.0  2021-02-08    peak  0.78       53
data_3  2021-02-17  2021-02-18  2.0  2021-02-22  valley  0.79       57
data_4  2021-03-03  2021-04-08  1.0  2021-04-09    peak  0.67        1
data_5  2021-03-17  2021-03-17  2.0  2021-03-19  valley  0.65       70
data_6  2021-03-31  2021-04-07 -4.0  2021-03-31  valley   0.2        3
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  0.67       70
data_8  2021-04-29  2021-04-30  2.0  2021-05-04  valley  0.48       69
data_9  2021-05-13  2021-05-14  0.0  2021-05-14    peak  0.41       66
data_10 2021-05-27  2021-05-28  2.0  2021-06-02    peak  0.38       66
data_11 2021-06-11  2021-06-15 -1.0  2021-06-14    peak   0.4       62
data_12 2021-06-25  2021-06-25  5.0  2021-07-02    peak  

In [146]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 1
n_harm_upper_limit = 70
fit_method = 'mean'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.65
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  0.04       35
data_1  2021-01-19  2021-01-21  2.0  2021-01-25  valley  0.08       25
data_2  2021-02-02  2021-02-08  2.0  2021-02-10  valley   0.0        3
data_3  2021-02-17  2021-02-19  2.0  2021-02-23  valley   0.0       33
data_4  2021-03-03  2021-03-18 -1.0  2021-03-17    peak   0.0        4
data_5  2021-03-17  2021-03-22 -3.0  2021-03-17    peak   0.0       13
data_6  2021-03-31  2021-04-05  1.0  2021-04-06    peak   0.0       12
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  0.02       67
data_8  2021-04-29  2021-04-30  2.0  2021-05-04  valley   0.0       68
data_9  2021-05-13  2021-06-09  1.0  2021-06-10    peak   0.0        5
data_10 2021-05-27  2021-05-28  1.0  2021-06-01    peak   0.0       57
data_11 2021-06-11  2021-06-15  0.0  2021-06-15    peak   0.0       51
data_12 2021-06-25  2021-07-15 -1.0  2021-07-14    peak  

In [147]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 1
n_harm_upper_limit = 70
fit_method = 'mean'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.58
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley   0.0       22
data_1  2021-01-19  2021-05-10 -1.0  2021-05-07    peak   0.0        1
data_2  2021-02-02  2021-02-22  0.0  2021-02-22  valley   0.0        8
data_3  2021-02-17  2021-03-10 -4.0  2021-03-04  valley   0.0        2
data_4  2021-03-03  2021-03-04  0.0  2021-03-04  valley  0.02       57
data_5  2021-03-17  2021-03-17  2.0  2021-03-19  valley   0.0       48
data_6  2021-03-31  2021-04-08  4.0  2021-04-14  valley   0.0        4
data_7  2021-04-15  2021-04-21 -1.0  2021-04-20  valley   0.0       17
data_8  2021-04-29  2021-05-24  0.0  2021-05-24    peak   0.0        3
data_9  2021-05-13  2021-05-13  0.0  2021-05-13  valley   0.0       28
data_10 2021-05-27  2021-06-16 -2.0  2021-06-14    peak   0.0        9
data_11 2021-06-11  2021-06-15 -1.0  2021-06-14    peak   0.0       47
data_12 2021-06-25  2021-07-29  0.0  2021-07-29    peak  

In [148]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 1
n_harm_upper_limit = 70
fit_method = 'rmse'
pv_method = 'HL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  2.0
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  1.25       30
data_1  2021-01-19  2021-05-10 -1.0  2021-05-07    peak  0.58        1
data_2  2021-02-02  2021-02-05  3.0  2021-02-10  valley  1.01       27
data_3  2021-02-17  2021-06-04  0.0  2021-06-04    peak  0.89        3
data_4  2021-03-03  2021-03-04  0.0  2021-03-04  valley  1.04       35
data_5  2021-03-17  2021-03-22 -3.0  2021-03-17    peak  0.58        1
data_6  2021-03-31  2021-04-01 -1.0  2021-03-31  valley   1.0       31
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  1.14       37
data_8  2021-04-29  2021-05-03  1.0  2021-05-04  valley  1.13       33
data_9  2021-05-13  2021-05-13  4.0  2021-05-19  valley  0.97       31
data_10 2021-05-27  2021-05-27  2.0  2021-06-01    peak  0.87        3
data_11 2021-06-11  2021-06-18 -3.0  2021-06-15    peak  1.08       36
data_12 2021-06-25  2021-06-30  4.0  2021-07-07    peak  1

In [149]:
stock_name = "^GSPC"
date_predict_start = '2021-01-01'
data_range = 200
slide_range = 10
n_slide = 26
pv_range = 2
n_harm_lower_limit = 1
n_harm_upper_limit = 70
fit_method = 'rmse'
pv_method = 'CL'
harmonics, model, errors, best_fit_harm, best_error, first_date, lead, pv, result_table, final_error=  main_funtion(
    stock_name, date_predict_start, data_range, slide_range,
    n_slide, pv_range, n_harm_lower_limit, n_harm_upper_limit, fit_method, pv_method)


final_error =  1.92
            s_date      t_date lead    ans_date      pv error best_fit
data_0  2021-01-04  2021-01-06 -2.0  2021-01-04  valley  1.14       28
data_1  2021-01-19  2021-05-10 -1.0  2021-05-07    peak  0.82        1
data_2  2021-02-02  2021-02-05  3.0  2021-02-10  valley  1.12       12
data_3  2021-02-17  2021-06-04  0.0  2021-06-04    peak   1.1        3
data_4  2021-03-03  2021-04-08  1.0  2021-04-09    peak  0.82        1
data_5  2021-03-17  2021-03-19  1.0  2021-03-22    peak   1.0       12
data_6  2021-03-31  2021-04-07 -4.0  2021-03-31  valley  0.45        3
data_7  2021-04-15  2021-04-16  2.0  2021-04-20  valley  1.31       41
data_8  2021-04-29  2021-04-30  2.0  2021-05-04  valley  0.82       56
data_9  2021-05-13  2021-05-25 -1.0  2021-05-24    peak  0.77       56
data_10 2021-05-27  2021-06-01  1.0  2021-06-02    peak  0.81       41
data_11 2021-06-11  2021-06-15 -1.0  2021-06-14    peak  0.69       41
data_12 2021-06-25  2021-06-25  5.0  2021-07-02    peak  